# Infering our trained news classifier

## Importing Required Libraries

In [1]:
import pandas as pd

In [2]:
# importing test file

test = pd.read_csv("test.csv")

In [3]:
test

,id,data,label
0,1,प्रधानमन्त्री केपी ओलीले नेपाललाई ‘वन बेल्ट वन...,None
1,2,"﻿ १७ साउन, पाल्पा। पाल्पा जिल्लाबाट ३० लाखको अ...",None
2,3,"﻿ ११ साउन, काठमाडौं । काठमाडौ महानगरपालिकाले ग...",None
3,4,"कला, संगीत र साहित्यलाई भूगोलको सिमानाले बाँध्...",None
4,5,सशस्त्र प्रहरी महानिरीक्षक कोषराज वन्तले बारा ...,None
...,...,...,...
5665,5666,"भारतको सिलोङमा जन्मिए, दार्जिलिङमा हुर्किए र क...",None
5666,5667,माओवादीले राजनीतिमा जान चाहने आफ्ना लडाकुलाई '...,None
5667,5668,चार दिनदेखिको लगातार वर्षाले सतबाज-गोकुलेश्वर ...,None
5668,5669,"सीके लाल भन्दै थिए, 'लामो समयको ठहरावपछि नेपाल...",None


## Downloading the model from s3 bucket

In [5]:
import boto3

AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''

s3 = boto3.client('s3', aws_access_key_id= AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
with open('./model-aug13-news-classifier-iter-4-33K.zip', '+wb') as f:
    s3.download_fileobj("realpha-models-registry", 'model-aug13-news-classifier-iter-4-33K.zip', f)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## Inference

In [8]:
# installing tranformers library
!pip install -U transformers

In [9]:
len(X_test)

5670

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name = "model-aug13-news-classifier-iter-4-33K"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

batch_size = 10
for i in range(0, len(X_test), batch_size):
    test_data = X_test[i : i + batch_size] 
    batch = tokenizer(test_data, padding=True, truncation=True, max_length=512, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**batch)
        predictions = F.softmax(outputs.logits, dim=1)
        labels = torch.argmax(predictions, dim=1)
        
    # open file in write mode
    with open(r'labels_33K.txt', '+a') as fp:
        for item in labels:
            # write each item on a new line
            fp.write("%s\n" % item.item())
    print(f"{i}/ {len(X_test)}")


0/ 5670
10/ 5670
20/ 5670
30/ 5670
40/ 5670
50/ 5670
60/ 5670
70/ 5670
80/ 5670
90/ 5670
100/ 5670
110/ 5670
120/ 5670
130/ 5670
140/ 5670
150/ 5670
160/ 5670
170/ 5670
180/ 5670
190/ 5670
200/ 5670
210/ 5670
220/ 5670
230/ 5670
240/ 5670
250/ 5670
260/ 5670
270/ 5670
280/ 5670
290/ 5670
300/ 5670
310/ 5670
320/ 5670
330/ 5670
340/ 5670
350/ 5670
360/ 5670
370/ 5670
380/ 5670
390/ 5670
400/ 5670
410/ 5670
420/ 5670
430/ 5670
440/ 5670
450/ 5670
460/ 5670
470/ 5670
480/ 5670
490/ 5670
500/ 5670
510/ 5670
520/ 5670
530/ 5670
540/ 5670
550/ 5670
560/ 5670
570/ 5670
580/ 5670
590/ 5670
600/ 5670
610/ 5670
620/ 5670
630/ 5670
640/ 5670
650/ 5670
660/ 5670
670/ 5670
680/ 5670
690/ 5670
700/ 5670
710/ 5670
720/ 5670
730/ 5670
740/ 5670
750/ 5670
760/ 5670
770/ 5670
780/ 5670
790/ 5670
800/ 5670
810/ 5670
820/ 5670
830/ 5670
840/ 5670
850/ 5670
860/ 5670
870/ 5670
880/ 5670
890/ 5670
900/ 5670
910/ 5670
920/ 5670
930/ 5670
940/ 5670
950/ 5670
960/ 5670
970/ 5670
980/ 5670
990/ 5670
1000/ 5670


## Importing the inference labels from txt file

In [11]:
with open("labels_33K.txt") as file:
    listfile = []
    for line in file:
        listfile.append(line.strip())
    print(listfile)

['6', '1', '1', '4', '6', '7', '6', '6', '5', '6', '5', '6', '7', '1', '6', '6', '2', '6', '5', '5', '6', '2', '5', '6', '6', '5', '5', '6', '6', '2', '6', '6', '6', '5', '6', '6', '5', '6', '6', '5', '5', '5', '6', '6', '6', '6', '1', '6', '6', '6', '0', '2', '7', '6', '6', '5', '2', '6', '7', '6', '5', '6', '6', '5', '7', '5', '6', '6', '6', '5', '5', '5', '2', '6', '6', '6', '5', '7', '2', '6', '5', '5', '7', '6', '6', '6', '6', '7', '6', '6', '2', '6', '6', '6', '6', '7', '6', '7', '6', '5', '2', '6', '7', '6', '5', '2', '6', '6', '6', '6', '1', '6', '6', '7', '2', '6', '6', '3', '6', '3', '5', '6', '6', '6', '6', '6', '6', '4', '2', '6', '2', '6', '7', '6', '6', '6', '6', '5', '6', '5', '6', '6', '2', '6', '2', '6', '5', '6', '6', '6', '7', '6', '4', '6', '6', '6', '2', '6', '6', '5', '6', '6', '5', '5', '5', '6', '6', '2', '6', '6', '6', '5', '6', '0', '2', '2', '4', '4', '2', '6', '6', '5', '2', '6', '2', '5', '5', '5', '5', '6', '6', '6', '2', '5', '2', '6', '6', '5', '7', '3',

In [13]:
test['encoded_label'] = listfile

In [14]:
test

,id,data,label,encoded_label
0,1,प्रधानमन्त्री केपी ओलीले नेपाललाई ‘वन बेल्ट वन...,None,6
1,2,"﻿ १७ साउन, पाल्पा। पाल्पा जिल्लाबाट ३० लाखको अ...",None,1
2,3,"﻿ ११ साउन, काठमाडौं । काठमाडौ महानगरपालिकाले ग...",None,1
3,4,"कला, संगीत र साहित्यलाई भूगोलको सिमानाले बाँध्...",None,4
4,5,सशस्त्र प्रहरी महानिरीक्षक कोषराज वन्तले बारा ...,None,6
...,...,...,...,...
5665,5666,"भारतको सिलोङमा जन्मिए, दार्जिलिङमा हुर्किए र क...",None,4
5666,5667,माओवादीले राजनीतिमा जान चाहने आफ्ना लडाकुलाई '...,None,6
5667,5668,चार दिनदेखिको लगातार वर्षाले सतबाज-गोकुलेश्वर ...,None,6
5668,5669,"सीके लाल भन्दै थिए, 'लामो समयको ठहरावपछि नेपाल...",None,5


In [15]:
label_liist = ['bank',
'economy',
'entertainment',
'health',
'literature',
'opinion',
'politics',
'sports',
'tourism']

In [16]:
test['decoded_label'] = test['encoded_label'].apply(lambda x: label_liist[int(x)])

In [17]:
final_data = test[['id', 'decoded_label']]

In [18]:
final_data.columns = ['id', 'label']

## Saving our predicted dataset

In [19]:
import csv

final_data.to_csv("test_predict_iter4_33K.csv", index=False, quoting=csv.QUOTE_ALL)